<a href="https://colab.research.google.com/github/Elkin77/spatial_database/blob/main/taller_bd_geoespaciales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
import os
from google.colab import drive

In [ ]:
!apt install libsqlite3-mod-spatialite

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libsqlite3-mod-spatialite
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 2,092 kB of archives.
After this operation, 8,524 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libsqlite3-mod-spatialite amd64 5.0.1-2build2 [2,092 kB]
Fetched 2,092 kB in 0s (5,428 kB/s)
Selecting previously unselected package libsqlite3-mod-spatialite:amd64.
(Reading database ... 131015 files and directories currently installed.)
Preparing to unpack .../libsqlite3-mod-spatialite_5.0.1-2build2_amd64.deb ...
Unpacking libsqlite3-mod-spatialite:amd64 (5.0.1-2build2) ...
Setting up libsqlite3-mod-spatialite:amd64 (5.0.1-2build2) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
conn = sqlite3.connect('/content/db_small.sqlite')

In [ ]:
conn.enable_load_extension(True)

conn.execute("SELECT load_extension(?)", ["/usr/lib/x86_64-linux-gnu/mod_spatialite.so"])

query = """
SELECT name FROM sqlite_schema
WHERE
type ='table' AND
name NOT LIKE 'sqlite_%';
"""
results_df = pd.read_sql_query(query, conn)
results_df

,name
0,spatial_ref_sys
1,spatialite_history
2,geometry_columns
3,spatial_ref_sys_aux
4,views_geometry_columns
5,virts_geometry_columns
6,geometry_columns_statistics
7,views_geometry_columns_statistics
8,virts_geometry_columns_statistics
9,geometry_columns_field_infos


In [ ]:
# Consultas SQL
# 1. Obtenga la longitud total de las líneas de vías fluviales (waterways_lines)
query1 = '''
SELECT SUM(ST_Length(geometry)) AS total_length
FROM waterways_lines
WHERE fclass = 'river';
'''
results_df = pd.read_sql_query(query1, conn)
results_df

,total_length
0,139150.810907


In [ ]:
# 2. Cuente el área total agrupada por tipo de uso terreno, como nature_reserve , industrial o
# residential (landuse_polygons columna fclass)

query2 = '''
SELECT fclass,
       SUM(ST_Area(geometry)) AS total_area
FROM landuse_polygon
GROUP BY fclass;
'''
results_df = pd.read_sql_query(query2, conn)
results_df

,fclass,total_area
0,allotments,6.613733e+06
1,cemetery,1.977648e+06
2,commercial,2.072685e+06
3,farm,2.715710e+07
4,forest,4.988121e+07
5,grass,7.670969e+06
6,heath,8.173466e+04
7,industrial,2.255320e+07
8,meadow,8.910864e+06
9,military,4.363472e+06


In [ ]:
# 3. Obtenga los 10 edificios más cercanos al origen 0,0 ( buildings_points)

query3 = '''
SELECT name, ST_Distance(ST_Point(0,0), geometry) AS distance
FROM buildings_points
ORDER BY distance
LIMIT 10;
'''
results_df = pd.read_sql_query(query3, conn)
results_df

,name,distance
0,,603705.768627
1,,603722.430004
2,,603722.619104
3,,603809.702025
4,,603814.127098
5,,603862.111855
6,,603867.839494
7,,604032.716406
8,,604049.714290
9,,604058.104933


In [ ]:
# Generar CSV

waterways_length = pd.read_sql_query(query1, conn)
landuse_areas = pd.read_sql_query(query2, conn)
closest_buildings = pd.read_sql_query(query3, conn)

# resultados en archivos CSV
pd.read_sql_query(query1, conn).to_csv('waterways_length.csv', index=False)
pd.read_sql_query(query2, conn).to_csv('landuse_areas.csv', index=False)
pd.read_sql_query(query3, conn).to_csv('closest_buildings.csv', index=False)

# Comprimir archivos CSV
!zip output_files.zip waterways_length.csv landuse_areas.csv closest_buildings.csv

  adding: waterways_length.csv (stored 0%)
  adding: landuse_areas.csv (deflated 34%)
  adding: closest_buildings.csv (deflated 40%)


In [ ]:
pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.9 MB/s eta 0:00:00


In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [ ]:
uri = "uri-mongo"
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
import pandas as pd
db=client.sample_geospatial
collection = db['shipwrecks']
cursor = collection.find({})
df = pd.DataFrame(list(cursor))
df
#df.to_csv()

,_id,recrd,vesslterms,feature_type,chart,latdec,londec,gp_quality,depth,sounding_type,history,quasou,watlev,coordinates
0,578f6fa2df35c7fbdbaed8c4,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.354779,-79.908127,,0.0,,,,always dry,"[-79.9081268, 9.3547792]"
1,578f6fa2df35c7fbdbaed8c5,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.334030,-79.935722,,0.0,,,,always dry,"[-79.9357223, 9.3340302]"
2,578f6fa2df35c7fbdbaed8c6,,,"Wrecks - Submerged, dangerous","US,U1,graph,DNC H1409860",9.356057,-79.907417,,0.0,,,depth unknown,always under water/submerged,"[-79.9074173, 9.3560572]"
3,578f6fa2df35c7fbdbaed8c7,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.339050,-79.913712,,0.0,,,,always dry,"[-79.9137115, 9.3390503]"
4,578f6fa2df35c7fbdbaed8c8,,,"Wrecks - Submerged, dangerous","US,U1,graph,DNC H1409860",9.341881,-79.910385,,0.0,,,depth unknown,always under water/submerged,"[-79.9103851, 9.3418808]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11090,578f6fa3df35c7fbdbaf0416,,,"Wrecks - Submerged, dangerous","US,US,graph,Chart 411",20.770311,-86.926767,,0.0,,,depth unknown,always under water/submerged,"[-86.926767, 20.770311]"
11091,578f6fa3df35c7fbdbaf0417,,,"Wrecks - Submerged, nondangerous","US,US,graph,Chart 411",26.311493,-93.334391,,0.0,,,depth unknown,always under water/submerged,"[-93.3343915, 26.311493]"
11092,578f6fa3df35c7fbdbaf0418,,,Wrecks - Visible,"US,U1,reprt,WN 14/12",22.448397,-89.598419,,0.0,,,,always dry,"[-89.5984188, 22.4483965]"
11093,578f6fa3df35c7fbdbaf0419,,,"Wrecks - Submerged, nondangerous","US,US,reprt,8thCGD,LNM 51/06",26.815334,-90.586395,,0.0,,,depth unknown,always under water/submerged,"[-90.5863952, 26.8153338]"


In [ ]:
collection.create_index([("location", "2dsphere")])

'location_2dsphere'

In [ ]:
# Búsqueda por proximidad: Encuentra todos los puntos más cercanos a las coordenadas 9.3,-79.90
query = {
    "coordinates": {
        "$near": {
            "$geometry": {
                "type": "Point",
                "coordinates": [-79.90, 9.3]
            },
            "$maxDistance": 5000
        }
    }
}


In [ ]:
cursor = collection.find(query)

In [ ]:
df = pd.DataFrame(list(cursor))
df

,_id,recrd,vesslterms,feature_type,chart,latdec,londec,gp_quality,depth,sounding_type,history,quasou,watlev,coordinates
0,578f6fa2df35c7fbdbaed8c7,,,Wrecks - Visible,"US,U1,graph,DNC H1409860",9.339050,-79.913712,,0,,,,always dry,"[-79.9137115, 9.3390503]"
1,578f6fa2df35c7fbdbaed8c8,,,"Wrecks - Submerged, dangerous","US,U1,graph,DNC H1409860",9.341881,-79.910385,,0,,,depth unknown,always under water/submerged,"[-79.9103851, 9.3418808]"


In [ ]:
# Guardar los resultados en un archivo CSV
df.to_csv('nearby_points.csv', index=False)

In [ ]:
# Búsqueda por área: Sugiera un polígono para describir la Costa Este de los estados unidos. Encuentre los puntos que están en esa área sugerida.
query = {
    "coordinates": {
        "$geoWithin": {
            "$geometry": {
                "type": "Polygon",
                "coordinates": [
                  [
                    [-79.0, 24.0],  # Florida
                    [-79.0, 44.0],  # Maine
                    [-66.0, 44.0],  # Maine
                    [-66.0, 24.0],  # Florida
                    [-79.0, 24.0]   # Florida (cierra el polígono)
                  ]
                ]}
        }
    }
}

In [ ]:
cursor = collection.find(query)

In [ ]:
df = pd.DataFrame(list(cursor))
df

,_id,recrd,vesslterms,feature_type,chart,latdec,londec,gp_quality,depth,sounding_type,history,quasou,watlev,coordinates
0,578f6fa3df35c7fbdbaef139,,,Wrecks - Visible,"US,US,graph,Chart 11534",33.766572,-78.816150,,0.0,,,,always dry,"[-78.8161503, 33.766572]"
1,578f6fa3df35c7fbdbaef13b,,,"Wrecks - Submerged, nondangerous","US,US,graph,Chart 11534",33.867940,-78.517886,,0.0,,,depth unknown,always under water/submerged,"[-78.5178855, 33.8679403]"
2,578f6fa3df35c7fbdbaef120,,,"Wrecks - Submerged, dangerous","US,US,graph,Chart 11534",33.834996,-78.546788,,0.0,,,depth unknown,always under water/submerged,"[-78.5467877, 33.8349964]"
3,578f6fa3df35c7fbdbaef121,,,"Wrecks - Submerged, dangerous","US,US,graph,Chart 11534",33.839986,-78.551598,,0.0,,,depth unknown,always under water/submerged,"[-78.5515981, 33.8399861]"
4,578f6fa3df35c7fbdbaef11f,,,"Wrecks - Submerged, dangerous","US,US,graph,Chart 11534",33.833280,-78.550030,,0.0,,,depth unknown,always under water/submerged,"[-78.5500304, 33.8332797]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,578f6fa3df35c7fbdbaee6c4,,,Wrecks - Visible,"US,US,graph,Chart 13292",43.668883,-70.203184,,0.0,,,,always dry,"[-70.203184, 43.668883]"
4362,578f6fa3df35c7fbdbaee6c3,,,Wrecks - Visible,"US,US,graph,Chart 13292",43.669165,-70.202912,,0.0,,,,always dry,"[-70.2029119, 43.6691645]"
4363,578f6fa3df35c7fbdbaee6c9,,,"Wrecks - Submerged, dangerous","US,US,graph,H 12494",43.661712,-70.209847,,6.7,,,least depth known,always under water/submerged,"[-70.2098466, 43.6617116]"
4364,578f6fa3df35c7fbdbaee6c6,,,"Wrecks - Submerged, dangerous","US,US,reprt,DD-22136",43.650484,-70.209217,,0.9,,,least depth known,always under water/submerged,"[-70.2092169, 43.6504839]"


In [ ]:
# Guardar los resultados en un archivo CSV
df.to_csv('polygon_costa_este.csv', index=False)

In [ ]:
!pip install folium

import folium

In [ ]:
cursor = collection.find({}, {"_id": 0, "coordinates": 1}).limit(1000)

In [ ]:
mapa = folium.Map(location=[40, -75], zoom_start=5)

In [ ]:
for document in cursor:
    coordinates = document["coordinates"]
    folium.Marker(location=[coordinates[1], coordinates[0]]).add_to(mapa)

In [ ]:
mapa